#Querying Source

In [0]:
%sql
select * from pyspark_realtime.source.products;

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [0]:
df = spark.sql("select * from pyspark_realtime.source.products")

# Deduplicate
df = df.withColumn("dedup",row_number().over(Window.partitionBy("id").orderBy(desc("updatedDate"))))
df = df.filter(col('dedup') == 1).drop('dedup')
display(df)

## **UPSERTS**

In [0]:
# Creating Delta Object
from delta.tables import DeltaTable
from pyspark.sql import functions
from pyspark.sql.window import Window

#try:
if len(dbutils.fs.ls("/Volumes/pyspark_realtime/source/db_volume/products_sink/")) > 0:
    dlt_obj = DeltaTable.forPath(spark,"/Volumes/pyspark_realtime/source/db_volume/products_sink/")
    dlt_obj.alias("trg").merge(df.alias("src"),"src.id = trg.id").whenMatchedUpdateAll(condition = "src.updatedDate >= trg.updatedDate").whenNotMatchedInsertAll().execute()
    print("This is upserting now...")

#except:
else:
    df.write.format("delta").mode("Overwrite").save("/Volumes/pyspark_realtime/source/db_volume/products_sink/")
    print("This is overwriting now...")

In [0]:
%sql
select * from delta.`/Volumes/pyspark_realtime/source/db_volume/products_sink/`